## GATOR 🐊
RUNNING A DUMMY EXAMPLE WITH GATOR

In [1]:
# import packages
import gator as ga
import os

In [2]:
# set the working directory to set paths to the example data
cwd = os.getcwd()
image = cwd + '/data/image/6.tif'
quantification = cwd + '/data/quantification/unmicst-6_cellMask.csv'
segmentation = cwd + '/data/segmentation/6cellMask.tif'
cellOutlines = cwd + '/data/segmenation/6cellOutlines.tif' # non mandatory; just for visual purposes

## Step-1: Generate Thumbnails for Training Data

#### If one were to start from scratch, the first step would be to train a model to recognize the marker of interest. In this example the data contains 11 channels `DNA1, bg2b, bg3b, bg4b, ECAD_1, CD45, CD4, CD3D, CD8A, CD45R, KI67` and as we are not interested in training a model to recognize DNA or background (`DNA1, bg2b, bg3b, bg4b`), we will only train  `ECAD_1, CD45, CD4, CD3D, CD8A, CD45R & KI67`. 

#### To do so, the first step is to create examples of `postive` and `negative` cells for each marker of interest. To facilitate this process, we can use the `generateThumbnails` function in `GATOR`. Under the hood the function identifies the cells that has high and low expression of the marker of interest and cuts out small thumbnails from the image. The high and low thresholds can be adjusted using the `threshold` parameter. The default (`threshold=[2, 12, 88, 98]`) threshold is defined by low: 2-12 percentile; high: 88-98 percentile. This should work fine for most cases. 

In [17]:
ga.generateThumbnails (csvPath=quantification, 
                        imagePath=image, 
                        markers = ["ECAD_1", "CD45", "CD4", "CD3D", "CD8A", "CD45R", "KI67"], 
                        markerChannels = [4,5,6,7,8,9,10], 
                        outputDir=cwd,
                        transformation=True, 
                        maxThumbnails=50, 
                        localNorm=True,
                        x_coordinate='X_centroid', 
                        y_coordinate='Y_centroid',
                        threshold=[2, 12, 88, 98], 
                        windowSize=64)

Processing Marker: ECAD_1
Processing Marker: CD45
Processing Marker: CD4
Processing Marker: CD3D
Processing Marker: CD8A
Processing Marker: CD45R
Processing Marker: KI67
Mission Accomplished


#### The output from the above function will be stored under `GATOR/Thumbnails/All markers of interest`. 

##### There are a number of parameters that function need to provided as seen above. Detailed explanations are avaialable in the documentation. Briefly, the function takes in the single-cell table (`csvPath`) with X and Y coordinates, the full image (`imagePath`) and lastly a list of `markers` for which thumbnails need to be generated. Please note as the program does not know which channels in the image corresponds to the `markers`, hence, the channel number of the `markers` also need to be provided via `markerChannels`. The `markerChannels` follow 1-indexing convention- so the first channel is represented by the number `1`. 

#### You would have also notices that I have set `maxThumbnails=50`. This basically means that even if more than 50 cells passed the set high and low percentile threshold, only 50 random cells will be used to generate the thumbnails. I generally generate a minimum of `2000` cells. As this is for illustration purpose only, I have set it to `50`. 

#### Now that the thumbnails are generated, one would manually go through the `TruePos` folder and `TrueNeg` folder and move files around as necessary. If there are any truly negative thumbnails in the `TruePos` folder, move it to `PosToNeg` folder. Similarly, if there are any truly positive thumbnails in `TrueNeg` folder, move it to `NegToPos` folder. You will often notice that imaging artifacts are captured in the `TruePos` folder and there will also likely be a number of true positives in the `TrueNeg` folder as the field of view (64x64) is larger than what the program used to identify those thumbnails (just the centroids of single cells at the center of that thumbnail).  

#### While you are manually sorting the postives and negative thumbnails, please keep in mind that you are looking for high-confident positives and high-confident negatives. It is absolutely okay to delete off majority of the thumbnails that you are not confident about. This infact makes it easy and fast as you are looking to only keep only thumbnails that are readily sortable.

#### Lastly, I generally use a whole slide image to generate these thumbnails as there will be enough regions with high expression and no expression of the marker of interest. If you look at the thumbnails of this dummy example, you will notice that most thumbnails of `TrueNeg` for `ECAD_1` does contain some level of `ECAD` as there is not enough regions to sample from. 

### Step-1a (optional)

#### You might have noticed in the above example, I had set `localNorm=True`, by default it is set to `False`. This parameter essentially creates a mirror duplicate copy of all the thumbnails and saves it under a folder named `localNorm`. The difference being that each thumbnail is normalized to the maximum intensity pixel in that thumbnail. It helps me to visually sort out the true positives and negatives faster and more reliably. As we will not use the thumbnails in the `localNorm` for training the deep learning model, we want to make sure all the manual sorting that we did in the `localNorm` folder is copied over to the real training data. I have written an additional function to help with this. Any moving or deleting of files that you did in the `localNorm` folder will be copied over to the real training data.

I randomly shifted some files from `TruePos` -> `PosToNeg` and `TrueNeg` -> `NegToPos`   for   `CD3D and CD4` for the purpose of illustration. I also randomly deleted some files. 

In [3]:
# list of folders to copy settings from
copyFolder = [cwd + '/GATOR/Thumbnails/localNorm/CD3D',
              cwd + '/GATOR/Thumbnails/localNorm/CD4']
# list of folders to apply setting to
applyFolder = [cwd + '/GATOR/Thumbnails/CD3D',
              cwd + '/GATOR/Thumbnails/CD4']
# note: Every copyFolder should have a corresponding applyFolder. The order matters! 

# The function accepts the four pre-defined folders. If you had renamed them, please change it using the parameter below.
ga.cloneFolder (copyFolder, 
                applyFolder, 
                TruePos='TruePos', TrueNeg='TrueNeg', 
                PosToNeg='PosToNeg', NegToPos='NegToPos')

Processing: CD3D
Processing: CD4


#### If you head over to the training data thumbails you will notice that the files have been shifited around exactly as in the `localNorm` folder. 

## Step-2: Generate Masks for Training Data

#### To train the deep learning model, in addition to the raw thumbnails a mask is needed. The mask lets the model know where the cell is located. Ideally one would manually draw on the thumbnails to locate where the positive cells are, however for the pupose of scalability we will use automated approaches to generate the Mask for us. The following function will generate the mask and split the data into `training, validation and test` that can be directly fed into the deep learning algorithm.

In [3]:
thumbnailFolder = [cwd + '/GATOR/Thumbnails/CD3D',
                   cwd + '/GATOR/Thumbnails/CD4']
outputDir = cwd

# The function accepts the four pre-defined folders. If you had renamed them, please change it using the parameter below.
# If you had deleted any of the folders and are not using them, replace the folder name with `None` in the parameter.
ga.generateTraining (thumbnailFolder, 
                     outputDir, 
                     file_extension=None,
                     TruePos='TruePos', NegToPos='NegToPos',
                     TrueNeg='TrueNeg', PosToNeg='PosToNeg')

Processing: CD3D
Processing: CD4
Mission Accomplished


#### If you head over to `GATOR/TrainingData/`, you will notice that each of the supplied marker above will have a folder with the associated `training, validataion and test` data that is required by the deep-learning algorithm to generate the model. 